In [1]:
%matplotlib inline
# %matplotlib widget

# Analysis of subjects

In [2]:
import csv
import os
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set(rc={'figure.facecolor':'white'})

In [3]:
from demapper.code.utils.analyze import extract_dataset, plot_results, plot_limits
from demapper.code.utils.config_cme import ALL_DATASETS, FILTERS, get_plot_columns, extract_params_f



### Run for one dataset

In [5]:
PICKED_DATASET = 'cmev4euc'

datadir = ALL_DATASETS['ch8'][PICKED_DATASET]
stats_path = os.path.join(datadir, 'compute_stats-averaged.csv')
results_path = os.path.join(datadir, 'compute_degrees_from_TCM/combined-compute_degrees_from_TCM.csv')

filter_by = FILTERS[PICKED_DATASET][0]
df, max_values = extract_dataset(stats_path, results_path, filter_by, extract_params_f, has_sbj=False)

resdir = datadir
if len(FILTERS[PICKED_DATASET]) > 1:
    resdir = os.path.join(datadir, filter_by)
    os.makedirs(resdir, exist_ok=True)

print(max_values)
df.sample(5)

len(stats):  539
len(results):  539
{}


,Mapper,K,R,G,ChangePointsIndicesError,ChangePointsCount,assortativity-mean,assortativity-std,coverage_TRs-mean,coverage_TRs-std,...,degree_TRs_avg-mean,degree_TRs_avg-std,degree_TRs_entropy-mean,degree_TRs_entropy-std,distances_entropy-mean,distances_entropy-std,distances_max-mean,distances_max-std,hrfdur_stat-mean,hrfdur_stat-std
219,EucNeuMapper_28_220_34,28,220,34,9.857143,8.0,-0.036726,0.048154,0.967825,0.038650,...,83.401378,37.156836,6.908817,0.507282,3.024135,0.494365,23.111111,11.390857,0.091280,0.039381
328,EucNeuMapper_36_260_50,36,260,50,8.571429,8.0,-0.064762,0.124836,0.999399,0.000962,...,1777.849444,451.534446,9.562615,0.163344,1.659228,0.331847,6.500000,2.007339,0.346187,0.067882
457,EucNeuMapper_48_180_34,48,180,34,10.714286,8.0,-0.084566,0.085587,0.990003,0.014266,...,208.012889,79.247691,8.103008,0.352344,2.236421,0.324564,11.944444,4.683792,0.138080,0.056013
227,EucNeuMapper_28_260_38,28,260,38,9.428571,8.0,-0.036798,0.060692,0.988747,0.015338,...,120.377011,28.677830,7.515664,0.367054,2.759768,0.489873,18.388889,8.374604,0.094544,0.028910
299,EucNeuMapper_36_100_46,36,100,46,5.857143,8.0,-0.042472,0.019234,0.999891,0.000318,...,2215.538889,393.982985,9.653462,0.077854,1.048827,0.202728,2.944444,0.416176,0.848275,0.100217


In [6]:

target_metrics = ['ChangePointsIndicesErrorRev', 'ChangePointsCount', 'coverage_nodes-mean', 'coverage_nodes-std', 'hrfdur_stat-mean', 'hrfdur_stat-std', 'distances_entropy-mean', 'distances_entropy-std']
log_metrics = ['ChangePointsIndicesErrorRev']

df['ChangePointsIndicesErrorRev'] = df.apply(
    lambda x: 1.0 / x['ChangePointsIndicesError'] if x['ChangePointsIndicesError'] > 0 else 100,
    axis=1)
# df['TransitionBetweenessRev'] = df.apply(lambda x: 1.0 / (x['TransitionBetweeness'] + 1), axis=1)


plot_results(df, 'SBJs_RKG', [], 'R', 'K', 'G', target_metrics, log_metrics, resdir=resdir)
plot_results(df, 'SBJs_KRG', [], 'K', 'R', 'G', target_metrics, log_metrics, resdir=resdir)

# for sbj_group_name, sbj_group in tqdm(sbjs_map.items()):
#     plot_results(df, sbj_group_name, sbj_group, fixedV, indexV, colV, target_metrics, log_metrics, resdir=resdir)
    

In [6]:
# Plot limits

target_metrics = {
    'ChangePointsIndicesError': [0, 12],
    'coverage_nodes-mean': [0.7, 1.0],
    'hrfdur_stat-mean': [0.15, 1.0],
    'distances_entropy-mean': [2.0, 10000.0]
}

fixedV, indexV, colV = get_plot_columns(filter_by)

plot_limits(df, 'SBJs_RKG', [], 'R', 'K', 'G', target_metrics, resdir=resdir)


## Recompute for all analysis that we have

In [8]:
# TYPE='degrees_TRs'
TYPE='compute_degrees_from_TCM'

target_metrics_detailed = ['ChangePointsIndicesErrorRev', 'ChangePointsCount', 'coverage_nodes-mean', 'coverage_nodes-std', 'hrfdur_stat-mean', 'hrfdur_stat-std', 'distances_entropy-mean', 'distances_entropy-std']
target_metrics = ['ChangePointsIndicesErrorRev', 'coverage_nodes-mean', 'hrfdur_stat-mean', 'distances_entropy-mean']
log_metrics = ['ChangePointsIndicesErrorRev']

target_metrics_limits = {
    'ChangePointsIndicesError': [0, 12],
    'coverage_nodes-mean': [0.7, 1.0],
    'hrfdur_stat-mean': [0.15, 1.0],
    'distances_entropy-mean': [2.0, 10000.0]
}
# CH = 'ch10'

# for CH in ['ch7', 'ch8', 'ch10', 'ch11']:
for CH in ['ch7', 'ch8', 'ch10', 'ch11']:
    for dataset_name in ALL_DATASETS[CH].keys():
#         if dataset_name not in ['cmev3_fast', 'cmev4_fast', 'cmev4euc_fast', 'cmev4euc', 'cmev5', 'cmev5MH']:
        if dataset_name not in ['cmev5MH']:
#         if not ('v3' in dataset_name or 'v4' in dataset_name):
            continue
        print('======= Processing', dataset_name)
        datadir = ALL_DATASETS[CH][dataset_name]
        print(datadir)
        break
        stats_path = os.path.join(datadir, 'compute_stats-averaged.csv')
    #     results_path = os.path.join(datadir, 'compute_degrees_from_TCM/combined-compute_degrees_from_TCM.csv')
        results_path = os.path.join(datadir, '{0}/combined-{0}.csv'.format(TYPE))

        try:
            for filter_by in FILTERS[dataset_name]:
                resdir = datadir
                if len(FILTERS[dataset_name]) > 1:
                    resdir = os.path.join(datadir, filter_by)
                    os.makedirs(resdir, exist_ok=True)
                df, max_values = extract_dataset(stats_path, results_path, filter_by, extract_params_f, has_sbj=False)

                fixedV, indexV, colV = get_plot_columns(filter_by)
                df['ChangePointsIndicesErrorRev'] = df.apply(lambda x: 1.0 / x['ChangePointsIndicesError'] if x['ChangePointsIndicesError'] > 0 else 10, axis=1)

                plot_results(df, 'SBJs-{}'.format(TYPE), [], fixedV, indexV, colV, target_metrics, log_metrics, resdir=resdir)
                plot_results(df, 'SBJs-{}_detailed'.format(TYPE), [], fixedV, indexV, colV, target_metrics_detailed, log_metrics, resdir=resdir)
                plot_limits(df, 'SBJs-{}'.format(TYPE), [], fixedV, indexV, colV, target_metrics_limits, resdir=resdir)
        except Exception as err:
            print("Warning! Didn't process '{}' because:".format(dataset_name))
            print(err)
            raise err

======= Processing cmev5MH
/Users/dh/workspace/BDL/demapper/results/cme/ch7_mappers_cmev5MH.json
======= Processing cmev5MH
/Users/dh/workspace/BDL/demapper/results/cme/ch8_mappers_cmev5MH.json
======= Processing cmev5MH
/Users/dh/workspace/BDL/demapper/results/cme/ch10_mappers_cmev5MH.json
======= Processing cmev5MH
/Users/dh/workspace/BDL/demapper/results/cme/ch11_mappers_cmev5MH.json


## Analysis for one subject on modularity

In [17]:
PICKED_DATASET = 'cmev4euc'
CH = 'ch8'

datadir = ALL_DATASETS[CH][PICKED_DATASET]
stats_path = os.path.join(datadir, 'compute_stats-averaged.csv')
results_path = os.path.join(datadir, 'modularity-avg.csv')

filter_by = FILTERS[PICKED_DATASET][0]
df, max_values = extract_dataset(stats_path, results_path, filter_by, extract_params_f, has_sbj=False)

resdir = datadir
if len(FILTERS[PICKED_DATASET]) > 1:
    resdir = os.path.join(datadir, filter_by)
    os.makedirs(resdir, exist_ok=True)

print(max_values)
df.sample(5)

len(stats):  539
len(results):  539
{}


,Mapper,K,R,G,NrNodes-mean,NrNodes-std,CalModNodes-mean,CalModNodes-std,CalModTRs-mean,CalModTRs-std,...,degree_TRs_avg-mean,degree_TRs_avg-std,degree_TRs_entropy-mean,degree_TRs_entropy-std,distances_entropy-mean,distances_entropy-std,distances_max-mean,distances_max-std,hrfdur_stat-mean,hrfdur_stat-std
351,EucNeuMapper_40_140_30,40,140,30,391.722222,122.720536,0.240743,0.081256,0.007412,0.011374,...,146.774900,32.379793,7.765472,0.318248,2.296674,0.381649,11.722222,4.599304,0.180667,0.054694
373,EucNeuMapper_40_260_34,40,260,34,836.777778,146.592896,0.467271,0.076225,0.052367,0.055771,...,76.759400,18.841842,7.081464,0.349147,2.780774,0.380284,18.777778,6.629307,0.054579,0.020121
41,EucNeuMapper_12_300_50,12,300,50,678.833333,135.417112,0.354055,0.093189,0.038287,0.030604,...,215.320944,66.766407,7.989669,0.373724,2.505363,0.481377,14.166667,6.564163,0.264192,0.061511
16,EucNeuMapper_12_180_34,12,180,34,396.000000,88.364220,0.326581,0.073557,0.038635,0.025896,...,92.856761,21.816130,7.163037,0.375157,2.595409,0.438002,14.166667,5.992643,0.313415,0.062305
210,EucNeuMapper_28_180_26,28,180,26,334.777778,94.513961,0.444863,0.133762,0.105098,0.065392,...,15.763083,6.099674,4.894934,0.492435,3.267808,0.397841,20.555556,5.659165,0.046603,0.023959


In [18]:
df.columns

Index(['Mapper', 'K', 'R', 'G', 'NrNodes-mean', 'NrNodes-std',
       'CalModNodes-mean', 'CalModNodes-std', 'CalModTRs-mean',
       'CalModTRs-std', 'assortativity-mean', 'assortativity-std',
       'coverage_TRs-mean', 'coverage_TRs-std', 'coverage_nodes-mean',
       'coverage_nodes-std', 'degree_TRs_avg-mean', 'degree_TRs_avg-std',
       'degree_TRs_entropy-mean', 'degree_TRs_entropy-std',
       'distances_entropy-mean', 'distances_entropy-std', 'distances_max-mean',
       'distances_max-std', 'hrfdur_stat-mean', 'hrfdur_stat-std'],
      dtype='object')

In [22]:
target_metrics = ['NrNodes-mean', 'NrNodes-std', 'CalModNodes-mean', 'CalModNodes-std',
                  'CalModTRs-mean', 'CalModTRs-std', 'coverage_nodes-mean', 'coverage_nodes-std',
                  'hrfdur_stat-mean', 'hrfdur_stat-std', 'distances_entropy-mean', 'distances_entropy-std']

plot_results(df, 'mod_SBJs', [], 'R', 'K', 'G', target_metrics, [], resdir=resdir)


## Compute modularity plots for all 

In [26]:
# TYPE='degrees_TRs'
# TYPE='compute_degrees_from_TCM'

target_metrics_detailed = ['NrNodes-mean', 'NrNodes-std', 'CalModNodes-mean', 'CalModNodes-std',
                  'CalModTRs-mean', 'CalModTRs-std', 'coverage_nodes-mean', 'coverage_nodes-std',
                  'hrfdur_stat-mean', 'hrfdur_stat-std', 'distances_entropy-mean', 'distances_entropy-std']
target_metrics = ['NrNodes-mean', 'CalModNodes-mean', 'CalModTRs-mean', 'coverage_nodes-mean', 
                  'hrfdur_stat-mean', 'distances_entropy-mean']
log_metrics = []

target_metrics_limits = {
#     'ChangePointsIndicesError': [0, 12],
    'coverage_nodes-mean': [0.7, 1.0],
    'hrfdur_stat-mean': [0.15, 1.0],
    'distances_entropy-mean': [2.0, 10000.0]
}
CH = 'ch8'

for dataset_name in ALL_DATASETS[CH].keys():
#     if dataset_name not in ['cmev3', 'cmev4euc']:
#         continue
    if not ('v3' in dataset_name or 'v4' in dataset_name):
        continue
    print('======= Processing', dataset_name)
    datadir = ALL_DATASETS[CH][dataset_name]
    stats_path = os.path.join(datadir, 'compute_stats-averaged.csv')
    results_path = os.path.join(datadir, 'modularity-avg.csv')

    try:
        for filter_by in FILTERS[dataset_name]:
            resdir = datadir
            if len(FILTERS[dataset_name]) > 1:
                resdir = os.path.join(datadir, filter_by)
                os.makedirs(resdir, exist_ok=True)
            df, max_values = extract_dataset(stats_path, results_path, filter_by, extract_params_f, has_sbj=False)

            fixedV, indexV, colV = get_plot_columns(filter_by)
            plot_results(df, 'mod_SBJs-{}'.format(TYPE), [], fixedV, indexV, colV, target_metrics, log_metrics, resdir=resdir)
            plot_results(df, 'mod_SBJs-{}_detailed'.format(TYPE), [], fixedV, indexV, colV, target_metrics_detailed, log_metrics, resdir=resdir)
#             plot_limits(df, 'SBJs-{}'.format(TYPE), [], fixedV, indexV, colV, target_metrics_limits, resdir=resdir)
    except Exception as err:
        print("Warning! Didn't process '{}' because:".format(dataset_name))
        print(err)
        raise err

======= Processing cmev3
len(stats):  462
len(results):  462
======= Processing cmev4
len(stats):  539
len(results):  539
======= Processing cmev4euc
len(stats):  539
len(results):  539
======= Processing cmev3_fast
len(stats):  640
len(results):  640
======= Processing cmev4_fast
len(stats):  900
len(results):  900
======= Processing cmev4euc_fast
len(stats):  900
len(results):  900
